# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | Eunice | Nwaobi | EN | 4h45m | 33.33% |
| Member 2: | Albert | Fudzi | AF | 4h45m | 33.33% |
| Member 3: | Ama | Yiadom Boakye | AYB | 4h45m | 33.33% |
| Member 4: | Ahsan | Farid | AF | Leave of Absense | Leave of Absense |


---

## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

---

In [144]:
# some library imports
import pandas as pd

In [145]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [146]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe
pre = "/content/drive/MyDrive/ENGG680_LAB_ASSIGNMENTS/Lab_Assignment4/"
# uncomment and compelete below
cv_set = pd.read_csv(f"{pre}UrbanNav-HK_Whampoa-20210521_sensors.csv")

# TODO: Load UrbanNav-HK_TST-20210517_sensors.csv to 'test_set1' dataframe
test_set1 = pd.read_csv(f"{pre}UrbanNav-HK_TST-20210517_sensors.csv")

# TODO: Load 2020-03-14-16-45-35.csv to 'test_set2' dataframe
test_set2 = pd.read_csv(f"{pre}2020-03-14-16-45-35.csv")

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [147]:
# TODO: Display first two rows of cv_set
# (1 mark)
cv_set.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max,motion_state
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187,rest
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144,rest


In [148]:
# TODO: Load the info method of cv_set
# (1 mark)
cv_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min        1534 non-null 

In [149]:
test_set1.info()
test_set2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        783 non-null    float64
 1   ay_sum        783 non-null    float64
 2   az_sum        783 non-null    float64
 3   gx_sum        783 non-null    float64
 4   gy_sum        783 non-null    float64
 5   gz_sum        783 non-null    float64
 6   a_sum         783 non-null    float64
 7   g_sum         783 non-null    float64
 8   ax_var        783 non-null    float64
 9   ay_var        783 non-null    float64
 10  az_var        783 non-null    float64
 11  gx_var        783 non-null    float64
 12  gy_var        783 non-null    float64
 13  gz_var        783 non-null    float64
 14  a_var         783 non-null    float64
 15  g_var         783 non-null    float64
 16  ax_min        783 non-null    float64
 17  ay_min        783 non-null    float64
 18  az_min        783 non-null    

In [150]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)
row_cv_set, col_cv_set = cv_set.shape
row_test_set1, col_test_set1 = test_set1.shape
row_test_set2, col_test_set2 = test_set2.shape

print(f"'cv_set' has {row_cv_set} rows and {col_cv_set} columns")
print(f"'test_set1' has {row_test_set1} rows and {col_test_set1} columns")
print(f"'test_set2' has {row_test_set2} rows and {col_test_set2} columns")

'cv_set' has 1534 rows and 33 columns
'test_set1' has 783 rows and 33 columns
'test_set2' has 297 rows and 33 columns


In [151]:
# TODO: Display all columns in cv_set
# (1 mark)
cv_set.columns

Index(['ax_sum', 'ay_sum', 'az_sum', 'gx_sum', 'gy_sum', 'gz_sum', 'a_sum',
       'g_sum', 'ax_var', 'ay_var', 'az_var', 'gx_var', 'gy_var', 'gz_var',
       'a_var', 'g_var', 'ax_min', 'ay_min', 'az_min', 'gx_min', 'gy_min',
       'gz_min', 'a_min', 'g_min', 'ax_max', 'ay_max', 'az_max', 'gx_max',
       'gy_max', 'gz_max', 'a_max', 'g_max', 'motion_state'],
      dtype='object')

In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [152]:
# TODO - Load target vector into y
# (1 mark)
y = cv_set["motion_state"]
y.head(2)

,motion_state
0,rest
1,rest


From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [153]:
# TODO - Load all features from cv_set to X
# (2 marks)
X = cv_set.iloc[:, :32]
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [164]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [155]:
# TODO - Perform test/train split
# (1 mark)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1073, 32), (461, 32), (1073,), (461,))

Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [156]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)
pipe = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])

Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [157]:
# TODO - Setup grid set
# (4 marks)
param_grid = {"svc__C":[0.01, 0.1, 1 ,10,100],
              "svc__gamma":[0.001, 0.01, 0.1,1],
              "svc__kernel":["rbf"],
              "scaler":[StandardScaler(), RobustScaler(), None]}

In [158]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'scaler': [StandardScaler(), RobustScaler(), None],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [159]:
# TODO: Print the results from the grid search
# (2 marks)
resultlist = sorted(grid.cv_results_.keys())
print(resultlist)
print()
best_param =  grid.cv_results_["params"][grid.best_index_]
print("Best params:\n{best_param}\n".format(best_param=best_param))
train_scores = [max(grid.cv_results_["split0_train_score"]),
                max(grid.cv_results_["split1_train_score"]),
                max(grid.cv_results_["split2_train_score"]),
                max(grid.cv_results_["split3_train_score"]),
                max(grid.cv_results_["split4_train_score"]),
                ]
best_train_score = max(train_scores)
print("Best cross-validation train score: {best_train_score:.2f}".format(best_train_score=best_train_score))
valid_scores = [max(grid.cv_results_["split0_test_score"]),
                max(grid.cv_results_["split1_test_score"]),
                max(grid.cv_results_["split2_test_score"]),
                max(grid.cv_results_["split3_test_score"]),
                max(grid.cv_results_["split4_test_score"]),
                ]
best_valid_score = max(valid_scores)
print("Best cross-validation validation score: {best_valid_score:.2f}".format(best_valid_score=best_valid_score))

from sklearn.metrics import accuracy_score
y_pred_test = grid.predict(X_test)
test_score = accuracy_score(y_test, y_pred_test)
print("Test-set score: {test_score:.2f}".format(test_score=test_score))

['mean_fit_time', 'mean_score_time', 'mean_test_score', 'mean_train_score', 'param_scaler', 'param_svc__C', 'param_svc__gamma', 'param_svc__kernel', 'params', 'rank_test_score', 'split0_test_score', 'split0_train_score', 'split1_test_score', 'split1_train_score', 'split2_test_score', 'split2_train_score', 'split3_test_score', 'split3_train_score', 'split4_test_score', 'split4_train_score', 'std_fit_time', 'std_score_time', 'std_test_score', 'std_train_score']

Best params:
{'scaler': RobustScaler(), 'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Best cross-validation train score: 1.00
Best cross-validation validation score: 0.96
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [111]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
# (2 marks)
from sklearn.metrics import confusion_matrix as cm, classification_report as cr
print(f"Classification report: {cr(y_test, y_pred_test)}\n\nConfusion matrix: {cm(y_test, y_pred_test)}")

Classification report:               precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461


Confusion matrix: [[153   2   0]
 [ 10 192   5]
 [  0   3  96]]


___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

#### 1.2.4 - Answers
1. The fit of the model seems appropriate. The results are good and the confusion matrix illustrates that the model predicts the first and last cases well but migh have a little more trouble with predicting the central case (2nd row).
2.  Based on the results, the C and gamma seem appropriate. However, the best C was 100 so maybe having a range of 10 to 1000 might be more appropriate for the task at hand. Based on literature, "gamma defines how far the influence of a single training example reaches" ([Scikit-learn](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html)). The "C parameter trades off correct classification of training examples against maximization of decision function margin" ([Scikit-learn](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html)). Based on these definitions, we could modify the C range and that will likely affect our selection of gamma as well.

___

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [115]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)
y1 = test_set1["motion_state"]
X1 = test_set1.iloc[:, :32]
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [116]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 = test_set2["motion_state"]
X2 = test_set2.iloc[:, :32]
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [126]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)
y_pred_test1 = grid.predict(X1)
test_score1 = accuracy_score(y1, y_pred_test1)
print("test_set1 score: {test_score:.2f}".format(test_score=test_score1))


y_pred_test2 = grid.predict(X2)
test_score2 = accuracy_score(y2, y_pred_test2)
print("test_set2 score: {test_score:.2f}".format(test_score=test_score2))

test_set1 score: 0.92
test_set2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [118]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
# (1 marks)
print(f"Classification report: {cr(y1, y_pred_test1)}\n\nConfusion matrix: {cm(y1, y_pred_test1)}")

Classification report:               precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783


Confusion matrix: [[200  31   0]
 [  7 387  17]
 [  0   7 134]]


Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [128]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print(f"Classification report: {cr(y2, y_pred_test2, zero_division=True)}\n\nConfusion matrix: {cm(y2, y_pred_test2)}")

Classification report:               precision    recall  f1-score   support

        rest       1.00      0.00      0.00        16
    straight       1.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.75      0.33      0.13       297
weighted avg       0.81      0.25      0.10       297


Confusion matrix: [[  0   0  16]
 [  0   0 206]
 [  0   0  75]]


___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1. The *test_set1* performs well but the *test_set2* underperforms significantly. It doesn't generalize well to the second test set. This might be an example of overfitting. However, it may also be we've discovered a new case or weakness with our model.
2. The model may be overly complex and may not generalize well to the new set. The new set may have used different sensors, cars, or variables that the old set is unable to work with. This may have introduced variance that our model is simply unable to detect.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [129]:
# TODO - Load selected features from cv_set to X
# (1 marks)
X = cv_set[["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [131]:
# TODO - Load motion_state column (target vector) into y
y = cv_set["motion_state"]
y.head(2)

,motion_state
0,rest
1,rest


___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [133]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X, y, test_size=.3, random_state=42, stratify=y)
X_train_red.shape, X_test_red.shape, y_train_red.shape, y_test_red.shape
grid2 = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid2.fit(X_train_red, y_train_red)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'scaler': [StandardScaler(), RobustScaler(), None],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [160]:
# TODO: Print the results from the grid search

resultlist2 = sorted(grid2.cv_results_.keys())
print(resultlist2)
print()
best_param2 =  grid2.cv_results_["params"][grid2.best_index_]
print("Best params:\n{best_param}\n".format(best_param=best_param2))
train_scores2 = [max(grid2.cv_results_["split0_train_score"]),
                max(grid2.cv_results_["split1_train_score"]),
                max(grid2.cv_results_["split2_train_score"]),
                max(grid2.cv_results_["split3_train_score"]),
                max(grid2.cv_results_["split4_train_score"]),
                ]
best_train_score2 = max(train_scores2)
print("Best cross-validation train score: {best_train_score:.2f}".format(best_train_score=best_train_score2))
valid_scores2 = [max(grid2.cv_results_["split0_test_score"]),
                max(grid2.cv_results_["split1_test_score"]),
                max(grid2.cv_results_["split2_test_score"]),
                max(grid2.cv_results_["split3_test_score"]),
                max(grid2.cv_results_["split4_test_score"]),
                ]
best_valid_score2 = max(valid_scores2)
print("Best cross-validation validation score: {best_valid_score:.2f}".format(best_valid_score=best_valid_score2))

y_pred_test_red = grid2.predict(X_test_red)
test_score2 = accuracy_score(y_test_red, y_pred_test_red)
print("Test-set score: {test_score:.2f}".format(test_score=test_score2))

['mean_fit_time', 'mean_score_time', 'mean_test_score', 'mean_train_score', 'param_scaler', 'param_svc__C', 'param_svc__gamma', 'param_svc__kernel', 'params', 'rank_test_score', 'split0_test_score', 'split0_train_score', 'split1_test_score', 'split1_train_score', 'split2_test_score', 'split2_train_score', 'split3_test_score', 'split3_train_score', 'split4_test_score', 'split4_train_score', 'std_fit_time', 'std_score_time', 'std_test_score', 'std_train_score']

Best params:
{'scaler': RobustScaler(), 'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Best cross-validation train score: 0.97
Best cross-validation validation score: 0.86
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [135]:
# TODO - Display confusion matrix and print classification reports
print(f"Classification report: {cr(y_test_red, y_pred_test_red)}\n\nConfusion matrix: {cm(y_test_red, y_pred_test_red)}")

Classification report:               precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461


Confusion matrix: [[154   1   0]
 [ 20 173  14]
 [  1  44  54]]


___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
1. The model in *1.2.3* is a better fit to its training set than *2.1.3* to its training set. Also, it is better on its test set (same distributions).

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [136]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)
y1 = test_set1["motion_state"]
X1 = test_set1[["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [137]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2 = test_set2["motion_state"]
X2 = test_set2[["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [138]:
# TODO - Print test score accuracies for test_set1 and test_set2
y_pred_test_red1 = grid2.predict(X1)
acc_1 = accuracy_score(y1, y_pred_test_red1)
print("test_set1 score: {accuracy:.2f}".format(accuracy=acc_1))

y_pred_test_red2 = grid2.predict(X2)
acc_2 = accuracy_score(y2, y_pred_test_red2)
print("test_set2 score: {accuracy:.2f}".format(accuracy=acc_2))

test_set1 score: 0.83
test_set2 score: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [139]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print(f"Classification report: {cr(y1, y_pred_test_red1)}\n\nConfusion matrix: {cm(y1, y_pred_test_red1)}")

Classification report:               precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783


Confusion matrix: [[229   2   0]
 [ 26 336  49]
 [  0  57  84]]


Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [140]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print(f"Classification report: {cr(y2, y_pred_test_red2)}\n\nConfusion matrix: {cm(y2, y_pred_test_red2)}")

Classification report:               precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297


Confusion matrix: [[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]


___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
1. The model generalizes well to both test sets (despite being lower than the first implementation for the first test set), especially considering it performed with 83% accuracy in the test set of same distribution.
2. The model from section 1 had a .96 accuracy on the test set from same distribution and a .92 for *test_set1* and .25 for *test_set2*, which is an average generilzation of .585. The model from section 2 had a .83 accuracy on the test set from same distribution and a .83 for *test_set1* and .84 for *test_set2*, which is an average generilzation of .835. Since our goal is maximize generalizaiton, model 2 is better. If our goal were to tailor our model to *test_set1* and the same distribtuion, we would answer the question differently.

____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)


**Answer**
1.   

In [ ]:
# TODO - Load group selected features from cv_set to X
# X =
X.head(2)

In [ ]:
# TODO - Load motion_state column (target vector) into y
# y =

___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`

___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1
# y1 =
# X1 =
X1.head(2)

In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2
# y2 =
# X2 =
X2.head(2)

___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
1.
1.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [ ]:
# TODO - Load all features from cv_set to X
# X =
X.head(2)

In [ ]:
# TODO - Load motion_state column (target vector) into y
# y =

___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`
# X_train, X_test, y_train, y_test =

In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [ ]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 marks)
# pipe =

Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)
# param_grid =

In [ ]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# y1 =
# X1 =
X1.head(2)

In [ ]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
# y2 =
# X2 =
X2.head(2)

___

#### 4.2.2 - Visualize Results

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1.

In [161]:
# TODO - Load group selected features from cv_set to X
X = cv_set[["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [162]:
# TODO - Load motion_state column (target vector) into y
y = cv_set["motion_state"]
y.head(2)

,motion_state
0,rest
1,rest


___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [167]:
# TODO - Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42, stratify=y)

In [168]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 mark)
pipe = Pipeline([("scaler", StandardScaler()), ("rf", RandomForestClassifier())])

In [201]:
# TODO - Define the parameter grid for your model
# (3 marks)

param_grid = {"rf__n_estimators":[10, 100],
              "rf__criterion":["gini", "entropy", "log_loss"],
              "rf__max_depth": [10, 20],
              "rf__max_leaf_nodes": [10, 20],
              "rf__max_features": ["sqrt", "log2", None],
              "rf__bootstrap": [True],
              "scaler":[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer(), None]}

In [202]:
# TODO - Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf', RandomForestClassifier())]),
             param_grid={'rf__bootstrap': [True],
                         'rf__criterion': ['gini', 'entropy', 'log_loss'],
                         'rf__max_depth': [10, 20],
                         'rf__max_features': ['sqrt', 'log2', None],
                         'rf__max_leaf_nodes': [10, 20],
                         'rf__n_estimators': [10, 100],
                         'scaler': [StandardScaler(), RobustScaler(),
                                    MinMaxScaler(), Normalizer(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [203]:
# TODO: Print the results from the grid search
resultlist = sorted(grid.cv_results_.keys())
print(resultlist)
print()
best_param =  grid.cv_results_["params"][grid.best_index_]
print("Best params:\n{best_param}\n".format(best_param=best_param))
train_scores = [max(grid.cv_results_["split0_train_score"]),
                max(grid.cv_results_["split1_train_score"]),
                max(grid.cv_results_["split2_train_score"]),
                max(grid.cv_results_["split3_train_score"]),
                max(grid.cv_results_["split4_train_score"]),
                ]
best_train_score = max(train_scores)
print("Best cross-validation train score: {best_train_score:.2f}".format(best_train_score=best_train_score))
valid_scores = [max(grid.cv_results_["split0_test_score"]),
                max(grid.cv_results_["split1_test_score"]),
                max(grid.cv_results_["split2_test_score"]),
                max(grid.cv_results_["split3_test_score"]),
                max(grid.cv_results_["split4_test_score"]),
                ]
best_valid_score = max(valid_scores)
print("Best cross-validation validation score: {best_valid_score:.2f}".format(best_valid_score=best_valid_score))

from sklearn.metrics import accuracy_score
y_pred_test = grid.predict(X_test)
test_score = accuracy_score(y_test, y_pred_test)
print("Test-set score: {test_score:.2f}".format(test_score=test_score))

['mean_fit_time', 'mean_score_time', 'mean_test_score', 'mean_train_score', 'param_rf__bootstrap', 'param_rf__criterion', 'param_rf__max_depth', 'param_rf__max_features', 'param_rf__max_leaf_nodes', 'param_rf__n_estimators', 'param_scaler', 'params', 'rank_test_score', 'split0_test_score', 'split0_train_score', 'split1_test_score', 'split1_train_score', 'split2_test_score', 'split2_train_score', 'split3_test_score', 'split3_train_score', 'split4_test_score', 'split4_train_score', 'std_fit_time', 'std_score_time', 'std_test_score', 'std_train_score']

Best params:
{'rf__bootstrap': True, 'rf__criterion': 'log_loss', 'rf__max_depth': 10, 'rf__max_features': None, 'rf__max_leaf_nodes': 20, 'rf__n_estimators': 100, 'scaler': RobustScaler()}

Best cross-validation train score: 0.91
Best cross-validation validation score: 0.89
Test-set score: 0.85


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [204]:
# TODO - Display confusion matrix and print classification report
print(f"Classification report: {cr(y_test, y_pred_test)}\n\nConfusion matrix: {cm(y_test, y_pred_test)}")

Classification report:               precision    recall  f1-score   support

        rest       0.95      0.95      0.95       155
    straight       0.80      0.90      0.85       207
        turn       0.79      0.58      0.67        99

    accuracy                           0.85       461
   macro avg       0.85      0.81      0.82       461
weighted avg       0.85      0.85      0.84       461


Confusion matrix: [[147   8   0]
 [  5 187  15]
 [  2  40  57]]


___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [205]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
y1 = test_set1["motion_state"]
X1 = test_set1[["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [206]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2 = test_set2["motion_state"]
X2 = test_set2[["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [207]:
# TODO - Print test score accuracies for test_set1 and test_set2
y_pred_test1 = grid.predict(X1)
test_score1 = accuracy_score(y1, y_pred_test1)
print("test_set1 score: {test_score:.2f}".format(test_score=test_score1))

y_pred_test2 = grid.predict(X2)
test_score2 = accuracy_score(y2, y_pred_test2)
print("test_set2 score: {test_score:.2f}".format(test_score=test_score2))

test_set1 score: 0.86
test_set2 score: 0.75


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [208]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print(f"Classification report: {cr(y1, y_pred_test1)}\n\nConfusion matrix: {cm(y1, y_pred_test1)}")

Classification report:               precision    recall  f1-score   support

        rest       0.98      0.99      0.98       231
    straight       0.84      0.89      0.87       411
        turn       0.66      0.53      0.59       141

    accuracy                           0.86       783
   macro avg       0.83      0.80      0.81       783
weighted avg       0.85      0.86      0.85       783


Confusion matrix: [[228   3   0]
 [  5 367  39]
 [  0  66  75]]


Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [209]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print(f"Classification report: {cr(y2, y_pred_test2)}\n\nConfusion matrix: {cm(y2, y_pred_test2)}")

Classification report:               precision    recall  f1-score   support

        rest       1.00      0.75      0.86        16
    straight       0.95      0.68      0.79       206
        turn       0.52      0.95      0.67        75

    accuracy                           0.75       297
   macro avg       0.82      0.79      0.77       297
weighted avg       0.84      0.75      0.76       297


Confusion matrix: [[ 12   4   0]
 [  0 140  66]
 [  0   4  71]]


___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
1. The RF with its optimal parameters fit better to the test set from the same distribution and *test_set1* than the *test_test2* (RESPECTIVELY, .85, .86, .75). The generalizability to new data is on average 0.805, which is lower than .835 in *section 2*. For our case, the SVC performs better than RF. R are better for larger datasets with high dimensionality. Given we simplified and selected our features, this is not the case. This is one possible cause of our observation. If we used the full feature list, we might expect to see robustness to overfitting and different importance values assigned to the different features.

____